# This code reads out various data sets and compares data availabily across several data sets

In [1]:
import os
import pandas as pd
import numpy as np
from pandas_datareader import wb
%load_ext autoreload
%autoreload 2

In [2]:
cwd = os.getcwd()
os.chdir("../git/wb_api_wrapper/")
from  wb_api_wrapper import * #Import functions to obtain World Bank data from the web and code to rename country names
os.chdir(cwd) #Get back to initial working directory

Get a dictionaty translating country spellings to the spelling usually used by the World Bank data sets

In [3]:
any_name_to_wb_name = pd.read_csv("../git/country_names/out/any_name_to_wb_name.csv",index_col="any",squeeze=True, encoding="utf-8")

The goal of this Notebook is to create a matrix of countries and the availability of indicators relevant for assessing resilience. We start off with loading some work which has been done already, contained in the file have_and_have_nots.csv.

In [4]:
data_matrix = pd.read_csv('have_and_have_nots.csv', index_col=0, squeeze=True, encoding="utf-8")

In [4]:
data_matrix.head(3)

,Code,Region,Income group,Lending category,HIPC,Data,missing_data
Economy,,,,,,,
Afghanistan,AFG,South Asia,Low income,IDA,HIPC,0,"plgp, avg_prod_k"
Albania,ALB,Europe & Central Asia,Upper middle income,IBRD,NaN,1,NaN
Algeria,DZA,Middle East & North Africa,Upper middle income,IBRD,NaN,0,"share1, social_p, social_r, avg_prod_k, share1..."


In [5]:
data_matrix.index #This provides and idea of how many countries are contained in data_matrix

Index(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       ...
       'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, RB', 'Vietnam',
       'Virgin Islands (U.S.)', 'West Bank and Gaza', 'Yemen, Rep.', 'Zambia',
       'Zimbabwe'],
      dtype='object', name='Economy', length=214)

Check if all the names in data_matrix are spelled according to WB spelling, and correct if not.

In [6]:
diff_data = data_matrix.index.difference(any_name_to_wb_name) # This command shows differences in the dataset
diff_data

Index(['Cabo Verde', 'Channel Islands', 'Curaçao', 'Côte d'Ivoire', 'Kosovo',
       'Sint Maarten (Dutch part)', 'São Tomé and Principe'],
      dtype='object', name='Economy')

In [7]:
diff_index = data_matrix.index.difference(any_name_to_wb_name.index) # This command shows differences in the index
diff_index

Index(['Channel Islands', 'Curaçao', 'Kosovo', 'Sint Maarten (Dutch part)'], dtype='object')

Names which appear in both commands 'data_matrix.index.difference(any_name_to_wb_name)' and 'data_matrix.index.difference(any_name_to_wb_name.index)' are missing from the dataset. Names which appear ONLY in 'data_matrix.index.difference(any_name_to_wb_name)' require renaming.

In [8]:
diff_data.difference(diff_index) #These entries need to be renamed.

Index(['Cabo Verde', 'Côte d'Ivoire', 'São Tomé and Principe'], dtype='object')

In [9]:
diff_index.difference(diff_data) # These entries are not in the country_name database.

Index([], dtype='object')

Translate the names in data_matrix.index so that they meet the World Bank type spelling

In [10]:
data_matrix = data_matrix.rename(index=any_name_to_wb_name) # This is a sweet one-line command to rename the index using a dictionary-like translation, in this case an_name_to_wb_name

# Now obtain further indicators from locally saved csv files. See the Notebook pull_data_wb

In [134]:
wb_data_inc20 = pd.read_csv('wb_data_raw/SI.DST.FRST.20.csv', encoding='utf-8').head(10)

In [135]:
wb_data_inc20

,country,value,year
0,Albania,8.85,2012
1,Algeria,6.96,1995
2,Angola,5.43,2008
3,Argentina,4.75,2013
4,Armenia,8.48,2013
5,Australia,7.09,2010
6,Austria,7.78,2012
7,Azerbaijan,13.37,2005
8,Bangladesh,8.89,2010
9,Belarus,9.65,2012


Now use the function mrv from wb_api_wrapper to keep only the latest entry and the year of the latest entry. This is done in order to simplify the DataFrame and thus make it easier to handle. The MultiIndex will be changed to an Index containing only the country names (in this case).

In [129]:
wb_data_inc20 = mrv(wb_data_inc20,include_year=True) #Careful, entries with only NaNs are dropped when using function mrv!

KeyError: 'country'

Now we also should rename the column labels before concatenating this table with data_matrix

In [20]:
wb_data_inc20.rename(columns={'value':'income share bottom quint.', 'year':'income share bottom quint: year'}, inplace=True)
wb_data_inc20['income share bottom quint.'] = wb_data_inc20['income share bottom quint.'] / 100 #Transform percentage to fraction
wb_data_inc20.head(2)

,income share bottom quint.,income share bottom quint: year
country,,
Albania,0.0885,2012
Algeria,0.0696,1995


Concatenate, i.e. add, the new data to the existing data matrix

In [11]:
data_matrix = pd.concat([data_matrix, wb_data_inc20], axis=1)
data_matrix.head(8)

NameError: name 'wb_data_inc20' is not defined

## Now load finance access data from FINDEX (Global Financial Inclusion Database)

In [128]:
wb_findex = pd.read_csv('wb_data_raw/FINDEX_selection.csv', index_col=0, encoding='utf-8')

Change the index to country

In [126]:
wb_findex

,year,WP11648.1,WP11648.8,WP15165.1,WP15165.8,WP_time_04.8
country,,,,,,
East Asia & Pacific (developing only),MRV,28.516100,15.334840,36.493060,26.096030,26.096030
East Asia & Pacific (developing only),2014,NaN,NaN,36.493060,26.096030,26.096030
East Asia & Pacific (developing only),2011,28.516100,15.334840,NaN,NaN,15.334840
Euro area,MRV,40.837310,40.125610,47.645660,37.590640,37.590640
Euro area,2014,NaN,NaN,47.645660,37.590640,37.590640
Euro area,2011,40.837310,40.125610,NaN,NaN,40.125610
Europe & Central Asia (developing only),MRV,4.909911,2.438396,8.368120,4.255587,4.255587
Europe & Central Asia (developing only),2014,NaN,NaN,8.368120,4.255587,4.255587
Europe & Central Asia (developing only),2011,4.909911,2.438396,NaN,NaN,2.438396


<bound method NDFrame.describe of                                                                                  country  \
country                                                                                    
East Asia & Pacific (developing only)              East Asia & Pacific (developing only)   
East Asia & Pacific (developing only)              East Asia & Pacific (developing only)   
East Asia & Pacific (developing only)              East Asia & Pacific (developing only)   
Euro area                                                                      Euro area   
Euro area                                                                      Euro area   
Euro area                                                                      Euro area   
Europe & Central Asia (developing only)          Europe & Central Asia (developing only)   
Europe & Central Asia (developing only)          Europe & Central Asia (developing only)   
Europe & Central Asia (developing only)       

In [127]:
mrv(wb_findex)

KeyError: 'year'

### Export the list of data availability to Excel and csv

In [ ]:
data_matrix.to_excel("output/data_availability.xlsx")
data_matrix.to_csv("output/data_availability.csv")

# Now select only IDA countries from data_matrix

Choose the IDA countries, which means dropping the non-IDA countries.
The cell below chooses all IDA countries, Blend countries and India and puts it into a new df called ida_countries

In [ ]:
ida_countries = data_matrix.loc[data_matrix['Lending category'] == "Blend"]
ida_countries = ida_countries.append(data_matrix.loc[data_matrix['Lending category'] == "IDA"])
ida_countries = ida_countries.append(data_matrix.loc['India'])
ida_countries = ida_countries.sort_index()

Drop some of the columns which are not necessary for the IDA comparison

In [ ]:
ida_countries.head(1)

In [ ]:
ida_countries.drop(['Income group', 'missing_data', 'Data'], axis=1, inplace=True)

In [ ]:
ida_countries.head(1)

Save the ida_countries file

Export the list of data availability to Excel and csv

In [ ]:
ida_countries.to_excel("output/IDA_countries_data_availability.xlsx")
ida_countries.to_csv("output/IDA_countries_data_availability.csv")